To Do:
-Cycle through directory to process all zip files
-Process all files in zip file
-Process second tab of Excel spreadsheet
-Load table of what items are associated with what items, trim table to only items created after 2015
-Determine where data stands
-Calculate cost per packet, divide cost per packet by number of items
-Scope out messy/unmatched data